In [2]:
spark

In [3]:
!pip install --upgrade colorama
from pysparkling import *
import h2o
hc = H2OContext.getOrCreate(spark)

Requirement already up-to-date: colorama in /opt/conda/lib/python3.6/site-packages
Connecting to H2O server at http://172.17.0.2:54321... successful.


H2O cluster uptime:,16 secs
H2O cluster version:,3.14.0.7
H2O cluster version age:,1 month
H2O cluster name:,sparkling-water-jovyan_local-1511264464675
H2O cluster total nodes:,1
H2O cluster free memory:,866 Mb
H2O cluster total cores:,1
H2O cluster allowed cores:,1
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://172.17.0.2:54321
H2O connection proxy:,None



Sparkling Water Context:
 * H2O name: sparkling-water-jovyan_local-1511264464675
 * cluster size: 1
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (driver,172.17.0.2,54321)
  ------------------------

  Open H2O Flow in browser: http://172.17.0.2:54321 (CMD + click in Mac OSX)

    


In [4]:
import pandas as pd
import numpy as Mnp
import pandas.io.sql as psql
# import pyodbc
import csv
import matplotlib.pyplot as plt
import seaborn as sns

### Seaborn style
sns.set_style("whitegrid")
pd.set_option('display.max_columns', None)

# pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.float_format', lambda x: '{:,.1f}'.format(x))

%matplotlib inline

In [5]:
df = sqlContext.read.parquet('/home/jovyan/work/sparkling-water-2.2.2/data/s3/flight.pq.11.comb')

In [6]:
df.count()

106453155

In [7]:
df_sample = df.sample(False, 0.000001, 42)

In [8]:
df_sample.count()

96

In [6]:
# !mkdir /home/jovyan/.local/h2o_jar
# !cp /home/jovyan/work/h2o-3.14.0.7/h2o.jar /home/jovyan/.local/h2o_jar
# # H2O

# # turn off proxy so that h2o can run properly
# import os
# # os.environ.pop("HTTP_PROXY")

# import h2o
# h2o.init(nthreads = -1)

In [9]:
%%time
hf = hc.as_h2o_frame(df_sample)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1035, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 883, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1040, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o43.asH2OFrame

In [ ]:
hf.head(5)

hf.describe()

hf.types

hf['leg1_carrierSummary_airlineName'].levels()

hf['depDayOfWeek'].levels()

# hf['JV_X_Excess'] = hf.interaction(['JV Description', 'Destination'], pairwise=False, max_factors=10, min_occurrence=500)

hf.columns

In [ ]:
features = ['price',
         'fromCity',
         'toCity', 
         'leadTime',
         'leg1_stops',
         'leg2_stops',
         'leg1_noOfTicketsLeft',
         'leg2_noOfTicketsLeft',
         'leg1_carrierSummary_airlineName',
         'leg2_carrierSummary_airlineName',
         'leg1_departureTime_hour',
         'leg2_departureTime_hour',
         'depWeekOfYear',
         'depDayOfWeek',
         'retWeekOfYear',
         'retDayOfWeek',
         'searchDayOfWeek',
         'leg1_cabinClass_0',
         'leg1_cabinClass_1',
         
         'leg2_cabinClass_0',
         'leg2_cabinClass_1',
         
         'trip'
         ]

target = 'priceWillDrop'

other_cols = [ 'futureMinPrice', 'saving','searchDate', 'routeCombKey','randVar',
             'leg1_cabinClass_2','leg2_cabinClass_2',]

features, target

hf.describe()

hf[hf['leg1_cabinClass_1'].isna(), 'leg1_cabinClass_1'] = 0 #missing
hf.describe()

hf[target] = hf[target].asfactor()

# target = targets[0]
# target

# hf2 = hf[targets + features2]

# # hf2[hf2[target] <= 0, target]=0
# for target in targets:
#     hf2[hf2[target] > 0, target]=1
#     hf2[hf2[target] <= 0, target]=0
#     hf2[target] = hf2[target].asfactor()
# # for target in targets:
# #     hf2[hf2[target] <= 0, target]=0

# hf2[hf2['Lead Time'] < 0, 'Lead Time']=0
# hf2[hf2['Lead Time'] > 366, 'Lead Time']=366
# hf2[hf2['Trip Length'] < 0, 'Trip Length']=0
# hf2[hf2['Trip Length'] > 366, 'Trip Length']=366
# hf2['excess'] = hf2['excess'].asfactor()
# hf2[hf2['oldest age'] < 15, 'oldest age']=15
# hf2[hf2['oldest age'] >= 90, 'oldest age']=90
# hf2[hf2['Traveller Count'] >= 3, 'Traveller Count']=3

df['searchDate'].value_counts()


import datetime

train=hf[hf['searchDate']<=datetime.datetime(2017, 6, 19, 0, 0, 0), :]
valid=hf[(hf['searchDate']>datetime.datetime(2017, 6, 19, 0, 0, 0)) & 
         (hf['searchDate']<=datetime.datetime(2017, 6, 21, 0, 0, 0)), :]
test=hf[hf['searchDate']>datetime.datetime(2017, 6, 21, 0, 0, 0), :]

train.shape, test.shape

train.nrow, valid.nrow, test.nrow

# splits = hf.split_frame(ratios=[0.7, 0.15], seed=1)  

# train = splits[0]
# valid = splits[1]
# test = splits[2]

# gbm_regressor.train(x=features, y=targets[0], training_frame=train)

# gbm_regressor.model_performance(test)

%%time

from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.automl import H2OAutoML

# print("############### Modelling: " + target + " ################")
gbm = H2OGradientBoostingEstimator(distribution="bernoulli", 
                                             ntrees=1, max_depth=1, min_rows=1000, learn_rate=1,
                                                stopping_rounds=10,
                                            stopping_tolerance=0.01, seed=0)
gbm.train(x=features, y=target,
          training_frame = train,
          validation_frame = valid)
gbm


# Run AutoML for 30 seconds
# aml = H2OAutoML(max_runtime_secs = 3600)
# aml.train(x=features, y=target,
#           training_frame = train,
#           validation_frame = valid,
#           leaderboard_frame = test)

# # View the AutoML Leaderboard
# lb = aml.leaderboard
# lb
# # # aml.leader    

aml.leader

important_features

hf[important_features]

%%time

imp_threshold = 0.01

print("############### Modelling: " + target + " ################")
model = aml.leader
print(model.confusion_matrix(valid=True))    
model.varimp_plot()   
important_features = [i[0] for i in model.varimp() if i[3]  > imp_threshold]
for f in important_features:        
    if f in ['price',
#              'leg1_carrierSummary_airlineName',
             'depWeekOfYear',
             'leadTime',
#              'leg2_carrierSummary_airlineName',
             'depDayOfWeek',
             'leg1_departureTime_hour',
             'toCity',
             'leg2_departureTime_hour',
             'searchDayOfWeek',
             'leg1_noOfTicketsLeft',
             'leg2_noOfTicketsLeft',
             'fromCity',
             'leg1_stops',
             'trip']:
#             model.partial_plot(data=valid,cols=[f],server=True, plot=True, nbins=272) 
#         else:
        model.partial_plot(data=test,cols=[f],server=True, plot=True, nbins=21)     #21    
model.plot()

model.download_mojo(path='D:\\flight.pq.11\\', get_genmodel_jar=True)